In [1]:
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from main import *
import random
from library.hill_climbing.hc import hill_climb
from library.simulated_annealing.sa import simulated_annealing

In [2]:
grid = np.zeros((10,10))
pieces = pieces_generator((10,10))
grid, pieces_coordinates = tetrimino_fitter(pieces, (5,5))

In [3]:
fig = go.Figure(
    data=[
        go.Scatter(
            x=pieces_coordinates[i][:, 0],
            y=pieces_coordinates[i][:, 1],
            marker_size=50,
            marker_symbol="square",
            mode="markers",
        )
        for i in range(len(pieces_coordinates))
    ],
    layout=dict(
        width=800,
        height=800,
        yaxis=dict(
            range=[-0.5, grid.shape[1]-.5],
            fixedrange=True,
        ),
        xaxis=dict(
            range=[-0.5, grid.shape[0]-.5],
            tickfont_size=12,
        ),
    ),
)
fig.write_html(f'graphs/test_{grid.size}_size.html')
fig.show()

In [4]:
grid_shape = (10,10)
individuals = [Individual(pieces_generator(grid_shape)) for i in range(1)]
valid_set = [l+str(n) for n in range(4) for l in ["I", "L", "J", "T", "S", "Z", "O"]]
pop = Population(individuals, "max", 1, valid_set, grid_shape)

In [5]:
hill_climb(pop, True)


ind.fitness = 80.0
ind_representation_str = 'I4J2I1I3J1Z1L1Z2L1L2L4Z4O2T3L1L3O4S1L1O4T4Z3T4S3Z2'
ind.len_max = 2, ind.number_n_visited = 0
best_n.fitness = 88.0 ;; ind.fitness = 80.0 ;; diff = -8.0
(best_n.fitness * m) = 88.0 > (ind.fitness * m) = 80.0

ind.fitness = 88.0
ind_representation_str = 'I4J2I1I3J1Z1L1Z2L1L4L2Z4O2T3L1L3O4S1L1O4T4Z3T4S3Z2'
ind.len_max = 13, ind.number_n_visited = 0
best_n.fitness = 88.0 ;; ind.fitness = 88.0 ;; diff = 0.0
Found plateau on ['I4', 'J2', 'I3', 'I1', 'J1', 'Z1', 'L1', 'Z2', 'L1', 'L4', 'L2', 'Z4', 'O2', 'T3', 'L1', 'L3', 'O4', 'S1', 'L1', 'O4', 'T4', 'Z3', 'T4', 'S3', 'Z2'] and ['I4', 'J2', 'I1', 'I3', 'J1', 'Z1', 'L1', 'Z2', 'L1', 'L4', 'L2', 'Z4', 'O2', 'T3', 'L1', 'L3', 'O4', 'S1', 'L1', 'O4', 'T4', 'Z3', 'T4', 'S3', 'Z2']

ind.fitness = 88.0
ind_representation_str = 'I4J2I3I1J1Z1L1Z2L1L4L2Z4O2T3L1L3O4S1L1O4T4Z3T4S3Z2'
ind.len_max = 13, ind.number_n_visited = 0
best_n.fitness = 88.0 ;; ind.fitness = 88.0 ;; diff = 0.0
Found plateau on ['I4', '

In [14]:
grid, pieces_coordinates = tetrimino_fitter(pop.individuals[0].representation, grid_shape)

In [15]:
fig = go.Figure(
    data=[
        go.Scatter(
            x=pieces_coordinates[i][:, 0],
            y=pieces_coordinates[i][:, 1],
            marker_size=50,
            marker_symbol="square",
            mode="markers",
        )
        for i in range(len(pieces_coordinates))
    ],
    layout=dict(
        width=800,
        height=800,
        yaxis=dict(
            range=[-0.5, grid.shape[1]-.5],
            fixedrange=True,
        ),
        xaxis=dict(
            range=[-0.5, grid.shape[0]-.5],
            tickfont_size=12,
        ),
    ),
)
fig.show()